In [1]:
import cv2
import sys,os
import numpy as np
# import rosbag
# import rospy
# from sensor_msgs.msg import Image
# from cv_bridge import CvBridge
# from pyquaternion import Quaternion
from numpy.linalg import inv, norm
from scipy.spatial.transform import Rotation as R, Slerp
from shutil import copyfile
import sqlite3



In [2]:
## modify colmap database 
database_file = "/home/pang/colmap_ws/practice1/hfnet/d.db"
IS_PYTHON3= True


def array_to_blob(array):
    if IS_PYTHON3:
        return array.tostring()
    else:
        return np.getbuffer(array)


def blob_to_array(blob, dtype, shape=(-1,)):
    if IS_PYTHON3:
        return np.fromstring(blob, dtype=dtype).reshape(*shape)
    else:
        return np.frombuffer(blob, dtype=dtype).reshape(*shape)

s1= {
    'front': {    
        'fx': 284.981,
        'fy': 286.102,
        'cx': 425.244,
        'cy': 398.468,
        'k1' : -7.3047108016908169e-03,
        'k2' : 4.3499931693077087e-02,
        'k3' : -4.1283041238784790e-02,
        'k4' : 7.6524601317942142e-03,
    } 
}

def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--database_file', required=True)
    args = parser.parse_args()
    return args


def update_db_intrinsics(db_connection, camera_id, fx,fy, cx, cy,k1,k2,k3,k4 ):
    cursor = db_connection.cursor()
    cursor.execute('SELECT params FROM cameras WHERE camera_id=?;',
                   (camera_id,))
    data = cursor.fetchall()
    assert len(data) == 1
    intrinsics = blob_to_array(data[0][0], np.double)

    # Update intrinsic values.
    intrinsics = np.array([fx,fy,cx,cy,k1,k2,k3,k4])


    cursor.execute('UPDATE cameras SET params = ?, model=? WHERE camera_id = ?;',
                   [array_to_blob(intrinsics),5, camera_id])

    cursor.close()



connection = sqlite3.connect(database_file)
cursor = connection.cursor()

# Get a mapping between image ids and image names and camera ids.
cursor.execute('SELECT image_id, camera_id, name FROM images;')
for image_id, camera_id, name in cursor:
    update_db_intrinsics(connection,1,**s1['front'])
    


cursor.execute('SELECT image_id, camera_id, name FROM images;')
for image_id, camera_id, name in cursor:
   
    cursor = connection.cursor()
    cursor.execute('UPDATE images SET camera_id = ? WHERE image_id = ?;',[1, image_id])
    cursor.close() 


cursor = connection.cursor()
cursor.execute('DELETE FROM cameras WHERE camera_id >= ?;',[2])
cursor.close()
cursor.close()
connection.commit()
connection.close()

/home/pang/.local/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  from ipykernel import kernelapp as app
